A biblioteca pandas fornece maneiras de se ler dados genéricos em tabelas de diversas origens, neste caso, importamos uma tabela de uma página web através da função pd.read_html(url). O resultado da leitura é um vetor que possui todas as tabelas da página, de onde vamos separar a tabela específica que queremos em um dataFrame.

O geopandas se trata de uma extensão do pandas, de modo que seus métodos são similares. O método gpd.read_file(file) busca nos arquivos da biblioteca um mapa especificado, nesse caso, um mapa mundi na projeção de Mercator.

In [5]:
import pandas as pd
import geopandas as gpd

url = 'https://en.wikipedia.org/wiki/List_of_countries_by_obesity_rate'
tabelas = pd.read_html(url)
dados = tabelas[0]

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

pd.set_option('display.max_rows', None)     # Mostrar todas as linhas
pd.set_option('display.max_columns', None)  # Mostrar todas as colunas
pd.set_option('display.width', None)        # Mostrar tudo alinhado 
pd.reset_option('display.max_rows')       #{
pd.reset_option('display.max_columns')    #{ Resetar as configurações
pd.reset_option('display.width')          #{

A biblioteca "fuzzywuzzy" fornece opções de comparação e manipulação de dados, permitindo que as linhas de diferentes colunas sejam comparadas, de modo que é possível gerar uma tabela que demonstra os nomes mais parecidos. Nesse caso, pequenas mudanças nos nomes de países podem ser encontradas (por exemplo os Estados Unidos, que podem constar como United States ou United States of America e a Costa do Marfim, que pode ser chamada de Ivory Coast ou Côte d'Ivorie).

In [6]:
from fuzzywuzzy import fuzz

pd.set_option('display.max_rows', None)  # Mostrar todas as linhas

dados_teste = dados.copy()
world_teste = world.copy()

dados_teste['Country'] = dados_teste['Country'].reset_index(drop=True)
world_teste['name'] = world_teste['name'].reset_index(drop=True)

linhas_extras_dados = dados_teste['Country'][~dados_teste['Country'].isin(world_teste['name'])]
linhas_extras_world = world_teste['name'][~world_teste['name'].isin(dados_teste['Country'])]

# Listas para armazenar os pares de correspondência
correspondencias = []

# Iterar pelos nomes dos países em uma das tabelas
for nome1 in linhas_extras_dados:
    melhor_correspondencia = None
    melhor_pontuacao = 0

    # Comparar com os nomes dos países na outra tabela
    for nome2 in linhas_extras_world:
        pontuacao = fuzz.ratio(nome1, nome2)

        # Atualizar a melhor correspondência se a pontuação for maior
        if pontuacao > melhor_pontuacao:
            melhor_pontuacao = pontuacao
            melhor_correspondencia = nome2

    # Adicionar a correspondência à lista de pares
    correspondencias.append((nome1, melhor_correspondencia))

# Criar um DataFrame com as correspondências
tabela_correspondencias = pd.DataFrame(correspondencias, columns=['Nome1', 'Nome2'])

# Imprimir a tabela de correspondências
print(tabela_correspondencias)

                               Nome1                     Nome2
0                              Nauru                 N. Cyprus
1                       Cook Islands              Falkland Is.
2                              Palau                 Palestine
3                   Marshall Islands              Falkland Is.
4                             Tuvalu                    Taiwan
5                               Niue                 Palestine
6                              Tonga                     Congo
7                              Samoa                   Myanmar
8                           Kiribati                  eSwatini
9     Federated States of Micronesia  United States of America
10                     United States  United States of America
11                           Bahrain                 W. Sahara
12                             Malta                   Myanmar
13                          Dominica            Dominican Rep.
14                Dominican Republic            Dominic

Continuando o trabalho da célula anterior, podemos substituir os nomes diferentes criando correspondências e utilizando .replace[]. Apesar de conveniente, ainda não é um método perfeito. Muitas nações pequenas ficam de fora, e certas áreas do mapa como "W. Sahara", precisam ser adicionadas à países. Temos também o problema de juntas nações que não se consideram a mesma, por exemplo Sérvia e Kosovo ou Israel e Palestina, porém muitas dessas disputas não interferem em processos de coleta de dados (como o censo nacional), de maneira que é possível juntar essas áreas por conveniência. Por fim, territórios não conectados como a Groenlândia e a Guiana Francesa, que possuem uma dinâmica populacional bem diferente, ainda ficam com os mesmos dados das nações principais. 

In [7]:
correspondencia = {
    "United States":"United States of America",
    "Israel":"Palestine",
    "Dominican Republic":"Dominican Rep.",
    "Czech Republic":"Czechia",
    "North Macedonia":"Macedonia",
    "Greenland":"Denmark",
    "Bosnia and Herzegovina":"Bosnia and Herz.",
    "Ivory Coast":"Côte d'Ivoire",
    "Equatorial Guinea":"Eq. Guinea",
    "Central African Republic":"Central African Rep.",
    "Democratic Republic of the Congo":"Dem. Rep. Congo",
    "South Sudan":"S. Sudan",
    "Servia":"Kosovo"
}

dados['Country'] = dados['Country'].replace(correspondencia)
world['name'] = world['name'].replace(correspondencia)

In [8]:
dados = world.merge(dados, how='left', left_on='name', right_on='Country')

sem_uso_ou_repetidas = ['gdp_md_est', 'iso_a3', 'continent', 'pop_est']

dados.drop(sem_uso_ou_repetidas, axis=1, inplace=True)

dados.dropna(axis=0, inplace=True, subset='Obesity rate (%)')

Por fim, usamos o folium para gerar o mapa e cruzar os dados.

In [10]:
import folium
from folium.plugins import MarkerCluster

mapa_obesidade = folium.Map()
folium.Choropleth(
    geo_data=dados,
    name='choropleth',
    data=dados,
    columns=['name', 'Obesity rate (%)'],
    key_on='feature.properties.name',
    fill_color='OrRd',
    fill_opacity=0.9,
    line_opacity=0.1,
    legend_name='Taxa de obesidade proporcional da população em %'
).add_to(mapa_obesidade)
mapa_obesidade.save('obes.html')